<a href="https://colab.research.google.com/github/FranciscoAnnoni/IA/blob/main/RedNeuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install kaggle 
!pip install -q kaggle 

In [ ]:
from google.colab import files 
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d franciscoannoni/caracters

In [ ]:
!unzip caracters.zip
!mkdir Midataset

%cd Midataset
!mkdir latino
!mkdir koreano
%cd ..

In [ ]:
!kaggle datasets download -d joonasyoon/charkor
!unzip charkor.zip

In [ ]:
from tensorflow.keras import datasets, layers, models
import numpy as np
import tensorflow as tf
import shutil
import os

# Eliminar la subcarpeta .ipynb_checkpoints
checkpoints_folder = '/content/Midataset/.ipynb_checkpoints'
if os.path.exists(checkpoints_folder):
    shutil.rmtree(checkpoints_folder)


class_names = ["koreano", "latino"]

# Generadores para sets de entrenamiento y pruebas
data_gen_entrenamiento = tf.keras.utils.image_dataset_from_directory(
    '/content/Midataset',
    validation_split=0.2,
    subset="training",
    color_mode='grayscale',
    seed=123,
    image_size=(250, 250),
    shuffle=True,
    batch_size=35,
    class_names=class_names
)

data_gen_validacion = tf.keras.utils.image_dataset_from_directory(
    '/content/Midataset',
    validation_split=0.2,
    subset="validation",
    color_mode='grayscale',
    seed=123,
    image_size=(250, 250),
    shuffle=True,
    batch_size=35,
    class_names=class_names
)

In [ ]:
#Imprimir 10 imagenes del generador de entrenamiento
import matplotlib.pyplot as plt
class_names = data_gen_entrenamiento.class_names
print(class_names)
plt.figure(figsize=(5, 5))
for images, labels in data_gen_entrenamiento.take(1):
    for i in range(10):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
        plt.xlabel(class_names[labels[i]])
        plt.show()


In [ ]:
modeloCNN2 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(250,250, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),

  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(2, activation='softmax')
])
modeloCNN2.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

historial=modeloCNN2.fit(data_gen_entrenamiento,epochs=2,batch_size=32,validation_data=data_gen_validacion)
finalModel = modeloCNN2.save("finalModel")

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import cv2
from PIL import Image

model = tf.keras.models.load_model('finalModel')

img = Image.open('/content/w.jpg')
img = np.array(img).astype(float)/255
img = cv2.resize(img, (250,250))
img = img.reshape(-1, 250, 250, 1)

predictions = model.predict(img)
resultado =np.argmax(predictions[0])

print(predictions)
print(resultado)
if resultado == 1:
  print("El caracter no es Koreano")
else:
  print("El caracter es del tipo " + "'" + class_names[resultado] +"'")

